In [40]:
import music21

In [41]:
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"

In [42]:
music21.converter.parse(little_star).show('mid')

#### 계이름과 숫자를 사아호 변환하는 표 - dic

In [63]:
temp1 = [1,2,3,4,5,6,7]
temp2 = ['c','d','e','f','g','a','b']
note2num = {}
for key,value in zip(temp2,temp1):
    note2num[key] = value
num2note = {}
for key,value in zip(temp1,temp2):
    num2note[key] = value

In [64]:
note2num

{'c': 1, 'd': 2, 'e': 3, 'f': 4, 'g': 5, 'a': 6, 'b': 7}

In [65]:
num2note

{1: 'c', 2: 'd', 3: 'e', 4: 'f', 5: 'g', 6: 'a', 7: 'b'}

#### ABC 표기를 시계열 데이터로 변환

In [66]:
def abc2timeseries(s):
    notes = s.split(' ')[2:]
    seq = []
    for i in notes:
        seq.append([note2num[i[0]],int(i[1])])
    return seq

#### 시계열 데이터를 ABC 표기로 변환

In [67]:
def timeseries2abc(t):
    s = 'tinynotation: 4/4'
    for i in t:
        s = s+ ' '+ num2note[i[0]]+str(i[1])
    return s

#### 원핫 코드로 변환하는 표

In [68]:
# 2음절,4음절,8음절
onehot = [[i,2] for i in range(1,8)] +[[i,4] for i in range(1,8)]+[[i,8] for i in range(1,8)]

#### 레이블을 원한 코드로 변환

In [69]:
import numpy as np
def to_onehot(la):
    t = []
    for i in range(len(la)):
        a = np.zeros(len(onehot))
        a[onehot.index(list(la[i]))] = 1.0
        t.append(a)
    return np.array(t)

In [70]:
# 시계열 데이터를 훈련 집합으로 자름

In [71]:
def seq2dataset(seq,window,horizon):
    X,Y = [],[]
    for i in range(len(seq) - (window+horizon)+1):
        x = seq[i:(i+window)]
        y = (seq[i+window+horizon-1])
        X.append(x); Y.append(y)
    return np.array(X,dtype=object), np.array(Y,dtype=object) # 경고메세지에따라서 dtype=object 추가

In [72]:
# 왼도우크기, 수평선계수
w = 8
h =  1

In [73]:
little_star

'tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2'

In [74]:
seq = abc2timeseries( little_star)
X,Y = seq2dataset(seq,w,h)
print(X.shape, Y.shape)
print(X[0], Y[0])

(34, 8, 2) (34, 2)
[[1 4]
 [1 4]
 [5 4]
 [5 4]
 [6 4]
 [6 4]
 [5 2]
 [4 4]] [4 4]


#### 훈련집합 구축

In [75]:
split = int(len(X)*1.0)  # 100%를 훈련 집합으로 사용
x_train = X[:split]; y_train = Y[:split]
y_train = to_onehot(y_train)

#### LSTM모델 설계와 학습

In [76]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [77]:
model = Sequential()
model.add(LSTM(units=128, activation='relu', input_shape=x_train[0].shape))
model.add(Dense(y_train.shape[1],activation='softmax')) # 분류문제
model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer='Adam',metrics=['accuracy'])
model.fit(x_train.astype(np.float32),y_train.astype(np.float32),epochs=200,batch_size=1, verbose=2)

Epoch 1/200
34/34 - 1s - loss: 2.8214 - accuracy: 0.0882 - 866ms/epoch - 25ms/step
Epoch 2/200
34/34 - 0s - loss: 2.3056 - accuracy: 0.2353 - 60ms/epoch - 2ms/step
Epoch 3/200
34/34 - 0s - loss: 2.0447 - accuracy: 0.2353 - 51ms/epoch - 1ms/step
Epoch 4/200
34/34 - 0s - loss: 2.0292 - accuracy: 0.3529 - 50ms/epoch - 1ms/step
Epoch 5/200
34/34 - 0s - loss: 1.9006 - accuracy: 0.2941 - 51ms/epoch - 1ms/step
Epoch 6/200
34/34 - 0s - loss: 1.8074 - accuracy: 0.3529 - 55ms/epoch - 2ms/step
Epoch 7/200
34/34 - 0s - loss: 1.6487 - accuracy: 0.4118 - 55ms/epoch - 2ms/step
Epoch 8/200
34/34 - 0s - loss: 1.5550 - accuracy: 0.3824 - 51ms/epoch - 2ms/step
Epoch 9/200
34/34 - 0s - loss: 1.6134 - accuracy: 0.3824 - 51ms/epoch - 1ms/step
Epoch 10/200
34/34 - 0s - loss: 1.4213 - accuracy: 0.4412 - 52ms/epoch - 2ms/step
Epoch 11/200
34/34 - 0s - loss: 1.3387 - accuracy: 0.5000 - 50ms/epoch - 1ms/step
Epoch 12/200
34/34 - 0s - loss: 1.2601 - accuracy: 0.5000 - 55ms/epoch - 2ms/step
Epoch 13/200
34/34 - 0s

Epoch 101/200
34/34 - 0s - loss: 0.0963 - accuracy: 0.9412 - 49ms/epoch - 1ms/step
Epoch 102/200
34/34 - 0s - loss: 0.0748 - accuracy: 0.9706 - 49ms/epoch - 1ms/step
Epoch 103/200
34/34 - 0s - loss: 0.0621 - accuracy: 0.9706 - 48ms/epoch - 1ms/step
Epoch 104/200
34/34 - 0s - loss: 0.1044 - accuracy: 0.9412 - 51ms/epoch - 1ms/step
Epoch 105/200
34/34 - 0s - loss: 0.0501 - accuracy: 0.9706 - 50ms/epoch - 1ms/step
Epoch 106/200
34/34 - 0s - loss: 0.0554 - accuracy: 0.9412 - 50ms/epoch - 1ms/step
Epoch 107/200
34/34 - 0s - loss: 0.0615 - accuracy: 0.9706 - 49ms/epoch - 1ms/step
Epoch 108/200
34/34 - 0s - loss: 0.0849 - accuracy: 0.9706 - 49ms/epoch - 1ms/step
Epoch 109/200
34/34 - 0s - loss: 0.1041 - accuracy: 0.9412 - 48ms/epoch - 1ms/step
Epoch 110/200
34/34 - 0s - loss: 0.0608 - accuracy: 0.9706 - 48ms/epoch - 1ms/step
Epoch 111/200
34/34 - 0s - loss: 0.0492 - accuracy: 0.9706 - 49ms/epoch - 1ms/step
Epoch 112/200
34/34 - 0s - loss: 0.0486 - accuracy: 0.9706 - 51ms/epoch - 2ms/step
Epoc

Epoch 199/200
34/34 - 0s - loss: 2.4351e-04 - accuracy: 1.0000 - 50ms/epoch - 1ms/step
Epoch 200/200
34/34 - 0s - loss: 2.3479e-04 - accuracy: 1.0000 - 67ms/epoch - 2ms/step


In [78]:
x_train.astype(np.float32)

array([[[1., 4.],
        [1., 4.],
        [5., 4.],
        [5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.]],

       [[1., 4.],
        [5., 4.],
        [5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.]],

       [[5., 4.],
        [5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.]],

       [[5., 4.],
        [6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.]],

       [[6., 4.],
        [6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.],
        [2., 4.]],

       [[6., 4.],
        [5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.],
        [2., 4.],
        [2., 4.]],

       [[5., 2.],
        [4., 4.],
        [4., 4.],
        [3., 4.],
        [3., 4.],
        [2., 4.],
        [2., 4.]

#### 학습된 모델로 편곡을 하는 함수(first_measure : 첫 소절, duration : 생성될 곡의 길이)

In [82]:
def arranging_music(model, first_measure,duration):
    music = first_measure
    for i in range(duration):
        p = model.predict(np.float32(np.expand_dims(music[-w:],axis=0)))
        music = np.append(music,[onehot[np.argmax(p)]],axis=0)
    return timeseries2abc(music)

In [89]:
new_song = arranging_music(model,x_train[0],50)
music21.converter.parse(new_song).show('mid')
print(new_song)

1/1 [==============================] - 0s 16ms/step


In [85]:
x_train[0]

array([[1, 4],
       [1, 4],
       [5, 4],
       [5, 4],
       [6, 4],
       [6, 4],
       [5, 2],
       [4, 4]], dtype=object)

In [88]:
# 임의로 아무거나 주기
dumysong = [[3, 4],
       [3, 4],
       [1, 2],
       [1, 2],
       [2, 8],
       [2, 8],
       [5, 2],
       [5, 4]]
new_song = arranging_music(model,dumysong,100)
music21.converter.parse(new_song).show('mid')
print(new_song)

1/1 [==============================] - 0s 24ms/step


---

### 세 곡을 시계열로 변환하고 결합

In [92]:
spring_picnic="tinynotation: 4/8 g8 e8 g8 e8 g8 a8 g4 e8 g8 e8 c8 d8 e8 c4 g8 e8 g8 e8 g8 a8 g4 b8 a8 g8 e8 d8 e8 c4"
butterfly="tinynotation: 2/4 g8 e8 e4 f8 d8 d4 c8 d8 e8 f8 g8 g8 g4 g8 e8 e8 e8 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4 d8 d8 d8 d8 d8 e8 f4 e8 e8 e8 e8 e8 f8 g4 g8 e8 e4 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4"
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"

In [97]:
seq1 = abc2timeseries(little_star)
seq2 = abc2timeseries(butterfly)
seq3 = abc2timeseries(spring_picnic)
seq = seq1+seq2+seq3

In [108]:
# seq는 순서대로 정렬되있기에 섞어줌
import pandas as pd
seq = pd.DataFrame(seq).sample(frac=1).values.tolist()

In [109]:
X,Y = seq2dataset(seq,w,h)

In [110]:
split = int(len(X)*1.0)
x_train, y_train = X[:split], Y[:split]
y_train = to_onehot(y_train)

In [111]:
model = Sequential()
model.add(LSTM(units=128, activation='relu', input_shape=x_train[0].shape))
model.add(Dense(y_train.shape[1],activation='softmax')) # 분류문제
model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer='Adam',metrics=['accuracy'])
model.fit(x_train.astype(np.float32),y_train.astype(np.float32),epochs=200,batch_size=1)

Epoch 1/200
116/116 [==============================] - 1s 2ms/step - loss: 2.7595 - accuracy: 0.1897
Epoch 2/200
116/116 [==============================] - 0s 2ms/step - loss: 2.5314 - accuracy: 0.2155
Epoch 3/200
116/116 [==============================] - 0s 2ms/step - loss: 2.5200 - accuracy: 0.2155
Epoch 4/200
116/116 [==============================] - 0s 2ms/step - loss: 2.5054 - accuracy: 0.2155
Epoch 5/200
116/116 [==============================] - 0s 2ms/step - loss: 2.4862 - accuracy: 0.2069
Epoch 6/200
116/116 [==============================] - 0s 2ms/step - loss: 2.4554 - accuracy: 0.2155
Epoch 7/200
116/116 [==============================] - 0s 2ms/step - loss: 2.4215 - accuracy: 0.2155
Epoch 8/200
116/116 [==============================] - 0s 2ms/step - loss: 2.3897 - accuracy: 0.1983
Epoch 9/200
116/116 [==============================] - 0s 2ms/step - loss: 2.3587 - accuracy: 0.1810
Epoch 10/200
116/116 [==============================] - 0s 2ms/step - loss: 2.3210 - accura

116/116 [==============================] - 0s 2ms/step - loss: 0.0178 - accuracy: 1.0000
Epoch 81/200
116/116 [==============================] - 0s 2ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 82/200
116/116 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 83/200
116/116 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 84/200
116/116 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 85/200
116/116 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 86/200
116/116 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 87/200
116/116 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 88/200
116/116 [==============================] - 0s 2ms/step - loss: 9.4967e-04 - accuracy: 1.0000
Epoch 89/200
116/116 [==============================] - 0s 2ms/step - loss: 7.9481e-04 - ac

116/116 [==============================] - 0s 2ms/step - loss: 9.6601e-08 - accuracy: 1.0000
Epoch 158/200
116/116 [==============================] - 0s 2ms/step - loss: 9.4545e-08 - accuracy: 1.0000
Epoch 159/200
116/116 [==============================] - 0s 2ms/step - loss: 8.4269e-08 - accuracy: 1.0000
Epoch 160/200
116/116 [==============================] - 0s 2ms/step - loss: 7.8103e-08 - accuracy: 1.0000
Epoch 161/200
116/116 [==============================] - 0s 2ms/step - loss: 7.3992e-08 - accuracy: 1.0000
Epoch 162/200
116/116 [==============================] - 0s 1ms/step - loss: 6.8854e-08 - accuracy: 1.0000
Epoch 163/200
116/116 [==============================] - 0s 2ms/step - loss: 6.1660e-08 - accuracy: 1.0000
Epoch 164/200
116/116 [==============================] - 0s 2ms/step - loss: 6.1660e-08 - accuracy: 1.0000
Epoch 165/200
116/116 [==============================] - 0s 1ms/step - loss: 5.2411e-08 - accuracy: 1.0000
Epoch 166/200
116/116 [============================

In [112]:
new_song = arranging_music(model,x_train[0],100)
music21.converter.parse(new_song).show('mid')

1/1 [==============================] - 0s 16ms/step
